In [1]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import configparser
from datetime import datetime

In [2]:
#Google Sheet OAth
def authenticate():
    # If modifying these scopes, delete the file token.pickle.
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

    """Shows basic usage of the Sheets API.
    Prints values from a sample spreadsheet.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('sheets', 'v4', credentials=creds)
    return service

service = authenticate()

In [3]:
def create_sheet(title):
    spreadsheet = {
        'properties': {
            'title': title
        }
    }
    spreadsheet = service.spreadsheets().create(body=spreadsheet,
                                        fields='spreadsheetId').execute()
    print('Spreadsheet ID: {0}'.format(spreadsheet.get('spreadsheetId')))
    return spreadsheet.get('spreadsheetId')


def get_data(spreadsheet_id, range_name='Sheet1'):
    result = service.spreadsheets().values().get(
        spreadsheetId=spreadsheet_id, range=range_name).execute()
    rows = result.get('values', [])
    return rows


def initiate_header(spreadsheet_id, range_name='Sheet1'):
    prev_data = len(get_data(spreadsheet_id))
    if prev_data > 0:
        return
    elif prev_data == 0:
        data = [['Person', 'Probability', 'Recorded Time']]

    body = {
        'values': [i for i in data]
    }

    service.spreadsheets().values().update(
        spreadsheetId=spreadsheet_id, range=range_name,
        valueInputOption='RAW', body=body).execute()

    
def upload_data(spreadsheet_id, data, range_name='Sheet1'):
    body = {
      'values': [i for i in data]
    }

    service.spreadsheets().values().append(
      spreadsheetId=spreadsheet_id, range=range_name,
      valueInputOption='RAW', body=body).execute()
    

def check_for_new_run(spreadsheet_name, config_file = 'smart_security_config.ini'):
    config = configparser.ConfigParser()
    if not os.path.exists(config_file):
        config['base'] = {'current_date': '', 'spreadsheet_id': ''}
    else:
        config.read(config_file)

    current_date = datetime.today().strftime("%d-%m-%Y")
    config_date = config['base']['current_date']
    if config_date.strip() == current_date.strip() and config['base']['spreadsheet_id'] is not '':
        print(
            "Script already ran today, not creating a new spreadsheet")
        spreadsheet_id = config['base']['spreadsheet_id']
        print(f"Spreadsheet ID: {spreadsheet_id}")
    else:
        print(
            f"Script last run on {config_date}. Date today: {current_date}")
        print("Creating new spreadsheet")
        spreadsheet_id = create_sheet(spreadsheet_name)
        config['base']['spreadsheet_id'] = spreadsheet_id
        config['base']['current_date'] = current_date

    # check if header exists and create otherwise
    initiate_header(spreadsheet_id)
    with open(config_file, 'w') as conf:
        config.write(conf)
    return spreadsheet_id

In [4]:
spreadsheet_name = "Smart Security Log %s" % (datetime.today().strftime("%d-%m-%Y"))
spreadsheet_id = check_for_new_run(spreadsheet_name)
print(f'https://docs.google.com/spreadsheets/d/{spreadsheet_id}')

Script already ran today, not creating a new spreadsheet
Spreadsheet ID: 1_SyfY2ZUM7JQ7SKKVa5sKAt9pz-nbB-gTlRV59OmbQ8
https://docs.google.com/spreadsheets/d/1_SyfY2ZUM7JQ7SKKVa5sKAt9pz-nbB-gTlRV59OmbQ8
